## Generate Word Cloud

In [2]:
import json
import re

from snowflake.snowpark import functions as fn
from snowflake.snowpark.functions import udf, col
from snowflake.snowpark.session import Session

# Create Snowflake session
session = Session.builder.configs(json.load(open("./creds.json"))).create()
session.clear_imports()
session.clear_packages()
session.add_packages("snowflake-snowpark-python")
session.add_packages("scikit-learn", "pandas", "numpy")

### Define UDF for extracting words

In [3]:
@udf(name='extract_words', is_permanent=False, stage_location='@models', replace=True)
def extract_words_udf(reviews: list) -> list:
    words = []
    for review in reviews:
        review_words = review.split()
        words.extend(review_words)
    return words

### Create Snowflake view VIEW_SENTIMENT_FLAG

In [4]:
view_sentiment_flag = session.sql("CREATE OR REPLACE VIEW PUBLIC.VIEW_SENTIMENT_FLAG AS \
            SELECT \
                REVIEW, \
                SENTIMENT, \
                CASE \
                    WHEN (SENTIMENT = 'positive') THEN 1 :: bigint \
                    ELSE 2 :: bigint \
                END AS SENTIMENT_FLAG \
            FROM \
                TRAIN_DATASET").collect()

### Get dataset from the view and extract words using UDF

In [1]:
words_dataset = session.sql('select REVIEW, SENTIMENT, SENTIMENT_FLAG, \
            extract_words(ARRAY_CONSTRUCT(REVIEW)) as extracted_words \
                                    from VIEW_SENTIMENT_FLAG')

words_dataset.show()

### Define UDF for cleaning words

In [2]:
@udf(name='clean_words', is_permanent=False, stage_location='@models', replace=True)
def clean_words(words: list) -> list:
    from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
    import string
    punctuation = set(string.punctuation)
    cleaned_words = []
    for word in words:
        if re.match(r'^[a-zA-Z]+$', word) and \
                word.lower() not in ENGLISH_STOP_WORDS and word.lower() not in punctuation:
            cleaned_words.append(word)
    return words

### Clean the words in the dataset using the UDF

In [3]:
cleaned_dataset = words_dataset.select(words_dataset.REVIEW, words_dataset.SENTIMENT, \
                                       words_dataset.SENTIMENT_FLAG, words_dataset.extracted_words,\
              fn.call_udf("clean_words", col("extracted_words")).alias('cleaned_words'))
cleaned_dataset.show()

### Calculate word frequencies for positive sentiment

In [4]:
positive_words_df = cleaned_dataset.filter(cleaned_dataset.SENTIMENT_FLAG == 1) \
    .select(fn.explode(cleaned_dataset.CLEANED_WORDS).alias("Word")) \
    .groupBy("Word") \
    .count() \
    .orderBy(fn.desc("count")) \
    .limit(100) \
    .select("Word", "count") \
    .write.mode('overwrite').saveAsTable('POSITIVE_WORDS_CLOUD')

### Calculate word frequencies for negative sentiment

In [5]:
negative_words_df = cleaned_dataset.filter(cleaned_dataset.SENTIMENT_FLAG == 2) \
    .select(fn.explode(cleaned_dataset.CLEANED_WORDS).alias("Word")) \
    .groupBy("Word") \
    .count() \
    .orderBy(fn.desc("count")) \
    .limit(100) \
    .select("Word", "count") \
    .write.mode('overwrite').saveAsTable('NEGATIVE_WORDS_CLOUD')

### Show the word frequency table for positive sentiment

In [6]:
session.sql("SELECT * FROM POSITIVE_WORDS_CLOUD").show()

In [7]:
session.sql("SELECT * FROM NEGATIVE_WORDS_CLOUD").show()

### Close the Snowflake session

In [8]:
session.close()